In [1]:
DRIVER_PATH = "G:\Deep Learning HPE\Age Detection Dataset\chromedriver.exe"
search_term = ["Adolescent boy"]
number_images = 100
target_path = './images'

In [2]:
from selenium import webdriver

In [3]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [4]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")
        

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [5]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [6]:
import os 
import time
import requests
import io
from PIL import Image
import hashlib

for ele in search_term:
    search_and_download(search_term = ele, driver_path=DRIVER_PATH, target_path=target_path, number_images=number_images)

Found: 100 search results. Extracting links from 0:100
Found: 101 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSVfJrBpyw6-xCNF6S61OGJAIbZ6SjO-m6K6g&usqp=CAU - as ./images\adolescent_boy\acd44be69d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQA8fby2VYc2hIsxhvR8gS8GAX41we6agmR9w&usqp=CAU - as ./images\adolescent_boy\be90bff617.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRr6-7b9lSj3B1ExlcNO8FG6gGInegxlXfmlA&usqp=CAU - as ./images\adolescent_boy\2470b827f4.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQLziYLrBXqWyfufqck_xfy6iFRYWFWRP5Y3g&usqp=CAU - as ./images\adolescent_boy\98b77b7d1a.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/portrait-positive-adolescent-boy-puberty-260nw-314476544.jpg - as ./images\adolescent_boy\5434050847.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQCXWxfXMQoaLaVXGNY3qlwy6joyF-IwhHgCA&usqp

SUCCESS - saved https://comps.canstockphoto.com/boy-adolescent-picture_csp41673572.jpg - as ./images\adolescent_boy\28779e5b6c.jpg
SUCCESS - saved https://www.unicef.org/sites/default/files/styles/hero_mobile/public/UN0149419.JPG?itok=Bf47efhF - as ./images\adolescent_boy\ba4c055c41.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQWwzjb5QjwndPNiWbc78OGBBCQW6lj_4tyQw&usqp=CAU - as ./images\adolescent_boy\8cce3b3e48.jpg
SUCCESS - saved https://www.hopkinsmedicine.org/-/media/images/health/3_-wellness/kids-and-teens-health/growing-child-adolescent-teaser.ashx - as ./images\adolescent_boy\aa48921773.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQgfGK1oVuHu3kPUCCw5zp9O5oCUcstF1XQAA&usqp=CAU - as ./images\adolescent_boy\ec179bb573.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQDIjIyywEf5rXbTxr6FP5Mhz71UJT1X-M2qQ&usqp=CAU - as ./images\adolescent_boy\d17bba723d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.